In [1]:
import pandas as pd 
import numpy as np
import re
from matplotlib import pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation


In [2]:
train = pd.read_csv('train_E6oV3lV.csv')
test = pd.read_csv('test_tweets_anuFYb8.csv')
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
id       31962 non-null int64
label    31962 non-null int64
tweet    31962 non-null object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [4]:
y_train = train.label.values

In [5]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Bidirectional, LSTM, GRU, Dense, Flatten, Input, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import Embedding, SpatialDropout1D, Dropout, BatchNormalization, Conv1D, concatenate, MaxPooling2D, AveragePooling2D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras import backend as K

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [6]:
X_train = train.tweet.values
X_test = test.tweet.values

In [7]:
max_feature = 40000
maxlen = 150
embedding_size = 300

In [8]:
print('Indexing word vectors')

#Glove Vectors
embeddings_index = {}
counter = 0
f = open('/home/paperspace/Desktop/Kaggle/Embeddings/glove.840B.300d.txt')
for line in f:
    #line = line.encode('ascii', 'replace')
    values = line.split()
    word = ''.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    #print(word)
#    counter = counter +1
    #print(values[1:])
    #if counter==2:
        #break
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_index))

Indexing word vectors
Total 2195892 word vectors.


In [9]:
#Regex to remove all Non-Alpha Numeric and space
special_character_removal=re.compile(r'[^a-z\d ]',re.IGNORECASE)

#regex to replace all numerics
replace_numbers=re.compile(r'\d+',re.IGNORECASE)

In [10]:
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    
    #Remove Special Characters
    text=special_character_removal.sub('',text)
    
#     text = re.sub('user', '', text)
    
    #Replace Numbers
    text=replace_numbers.sub('n',text)

    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [11]:
comments = []
for text in X_train:
    comments.append(text_to_wordlist(text, remove_stopwords=True))
    
test_comments=[]
for text in X_test:
    test_comments.append(text_to_wordlist(text, remove_stopwords=True))

In [12]:
tokenize = Tokenizer(num_words=max_feature)
tokenize.fit_on_texts(comments + test_comments)
X_train = tokenize.texts_to_sequences(comments)
X_test = tokenize.texts_to_sequences(test_comments)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')

In [13]:
word_index = tokenize.word_index
nb_words = min(max_feature, len(word_index))
embedding_matrix = np.zeros((nb_words, embedding_size))

In [14]:
# create a weight matrix for words in training docs
for word, i in word_index.items():
    if i >= max_feature: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [15]:
from sklearn.metrics import f1_score

class f1Evaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=1, batch_size = 512)
            y_pred[y_pred>=0.5]=1
            y_pred[y_pred<=0.5]=0
            score = f1_score(self.y_val, y_pred)
            print("\n F1 - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [16]:
def fmeasure(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [17]:
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [60]:
def get_LSTM_model():
    inp = Input((maxlen,))
    embed = Embedding(input_dim=max_feature, output_dim=embedding_size, weights = [embedding_matrix], trainable = True)(inp)
    x = SpatialDropout1D(0.2)(embed)
    x = Bidirectional(LSTM(256, return_sequences = True))(embed)
#     avg = GlobalAveragePooling1D()(x)
#     maxpool = GlobalMaxPooling1D()(x)
#     con = concatenate([avg, maxpool])
    x = Attention(maxlen)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inp, outputs=out)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[fmeasure])
    
    return model

In [42]:
def get_gru_model():
    inp = Input((maxlen,))
    embed = Embedding(input_dim=max_feature, output_dim=embedding_size, weights = [embedding_matrix], trainable = True)(inp)
    x = SpatialDropout1D(0.25)(embed)
    x = Bidirectional(GRU(124, return_sequences = True))(embed)
#     avg = GlobalAveragePooling1D()(x)
#     maxpool = GlobalMaxPooling1D()(x)
#     con = concatenate([avg, maxpool])
    x = Attention(maxlen)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inp, outputs=out)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[fmeasure])
    
    return model

In [43]:
model_gru = get_gru_model()
model_gru.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 150, 300)          12000000  
_________________________________________________________________
bidirectional_5 (Bidirection (None, 150, 248)          316200    
_________________________________________________________________
attention_5 (Attention)      (None, 248)               398       
_________________________________________________________________
dense_9 (Dense)              (None, 64)                15936     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 64)                256       
__________

In [61]:
model_lstm = get_LSTM_model()
model_lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 150, 300)          12000000  
_________________________________________________________________
bidirectional_8 (Bidirection (None, 150, 512)          1140736   
_________________________________________________________________
attention_8 (Attention)      (None, 512)               662       
_________________________________________________________________
dense_15 (Dense)             (None, 64)                32832     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 64)                256       
__________

In [81]:
# X_tra, X_val, y_tra, y_val = train_test_split(X_train, y_train, train_size=0.80, random_state=233)

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=111)
scores = []

In [20]:
batch_size = 1024
epochs = 50

X_tra, X_val, y_tra, y_val = train_test_split(X_train, y_train, train_size=0.80, random_state=233)

/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [102]:
# exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
# steps = int(len(X_tra)/batch_size) * epochs
# lr_init, lr_fin = 0.001, 0.0005
# lr_decay = exp_decay(lr_init, lr_fin, steps)
# K.set_value(model.optimizer.lr, lr_init)
# K.set_value(model.optimizer.decay, lr_decay)

In [24]:
model_gru.load_weights('bi_Gru.h5')
model_lstm.load_weights('bi_LSTM.hdf5')

In [82]:
y_pre = np.zeros((17197, 1))
k_split = 0
for tra, val in kfold.split(X_train, y_train):
    print('k-fold', k_split)
    model_lstm = get_LSTM_model()
    earlyStp = EarlyStopping(patience=5, monitor='val_fmeasure', mode = 'max', verbose=1)
    modelChePnt = ModelCheckpoint('bi_LSTM_attention.hdf5', save_best_only=True, verbose=1, monitor='val_fmeasure', mode='max')
    #     f1 = f1Evaluation(validation_data=(X_val, y_val), interval=1)

    hist = model_lstm.fit(x = X_train[tra], y = y_train[tra], epochs = 50, validation_data=[X_train[val], y_train[val]], batch_size=batch_size, callbacks=[earlyStp, modelChePnt])
    model_lstm.load_weights('bi_LSTM_attention.hdf5')
    
    y_pred_lstm = model_lstm.predict(X_test, batch_size=1024, verbose=1)
    y_pre = y_pre + y_pred_lstm
    k_split = k_split + 1

k-fold 0
Train on 25569 samples, validate on 6393 samples
Epoch 1/50
25569/25569 [==============================] - 28s 1ms/step - loss: 0.6450 - fmeasure: 0.2759 - val_loss: 5.3211 - val_fmeasure: 0.1311
Epoch 2/50
25569/25569 [==============================] - 24s 950us/step - loss: 0.4888 - fmeasure: 0.3660 - val_loss: 0.2496 - val_fmeasure: nan
Epoch 3/50
25569/25569 [==============================] - 25s 958us/step - loss: 0.3467 - fmeasure: 0.5290 - val_loss: 0.2445 - val_fmeasure: nan
Epoch 4/50
25569/25569 [==============================] - 25s 960us/step - loss: 0.2453 - fmeasure: 0.6539 - val_loss: 0.2043 - val_fmeasure: nan
Epoch 5/50
25569/25569 [==============================] - 25s 960us/step - loss: 0.1714 - fmeasure: 0.7458 - val_loss: 0.3614 - val_fmeasure: nan
Epoch 6/50
25569/25569 [==============================] - 25s 961us/step - loss: 0.1255 - fmeasure: nan - val_loss: 0.3464 - val_fmeasure: nan
Epoch 00006: early stopping
17197/17197 [===========================

25570/25570 [==============================] - 27s 1ms/step - loss: 0.0391 - fmeasure: 0.9213 - val_loss: 0.1273 - val_fmeasure: 0.6361
Epoch 10/50
25570/25570 [==============================] - 27s 1ms/step - loss: 0.0297 - fmeasure: 0.9414 - val_loss: 0.2324 - val_fmeasure: 0.6626
Epoch 11/50
25570/25570 [==============================] - 27s 1ms/step - loss: 0.0344 - fmeasure: 0.9347 - val_loss: 0.1333 - val_fmeasure: 0.6857
Epoch 12/50
25570/25570 [==============================] - 26s 1ms/step - loss: 0.0186 - fmeasure: 0.9657 - val_loss: 0.1511 - val_fmeasure: 0.5929
Epoch 13/50
25570/25570 [==============================] - 26s 1ms/step - loss: 0.0143 - fmeasure: 0.9741 - val_loss: 0.1504 - val_fmeasure: 0.6795
Epoch 14/50
25570/25570 [==============================] - 27s 1ms/step - loss: 0.0156 - fmeasure: 0.9720 - val_loss: 0.1343 - val_fmeasure: 0.7077
Epoch 15/50
25570/25570 [==============================] - 26s 1ms/step - loss: 0.0092 - fmeasure: 0.9840 - val_loss: 0.1797

In [62]:
earlyStp = EarlyStopping(patience=5, monitor='val_fmeasure', mode = 'max', verbose=1)
modelChePnt = ModelCheckpoint('bi_LSTM_attention.hdf5', save_best_only=True, verbose=1, monitor='val_fmeasure', mode='max')
# f1 = f1Evaluation(validation_data=(X_val, y_val), interval=1)

hist = model_lstm.fit(x = X_tra, y = y_tra, epochs = epochs, validation_data=[X_val, y_val], batch_size=batch_size, callbacks=[earlyStp, modelChePnt])
model_lstm.load_weights('bi_LSTM_attention.hdf5')

Train on 25569 samples, validate on 6393 samples
Epoch 1/50
25569/25569 [==============================] - 29s 1ms/step - loss: 0.6101 - fmeasure: 0.3575 - val_loss: 0.5247 - val_fmeasure: 0.4190
Epoch 2/50
25569/25569 [==============================] - 24s 945us/step - loss: 0.3479 - fmeasure: 0.6109 - val_loss: 0.4252 - val_fmeasure: 0.2751
Epoch 3/50
25569/25569 [==============================] - 25s 958us/step - loss: 0.2407 - fmeasure: 0.6639 - val_loss: 0.3153 - val_fmeasure: 0.5002
Epoch 4/50
25569/25569 [==============================] - 25s 966us/step - loss: 0.1667 - fmeasure: 0.7154 - val_loss: 0.3709 - val_fmeasure: 0.5793
Epoch 5/50
25569/25569 [==============================] - 25s 969us/step - loss: 0.1189 - fmeasure: 0.7694 - val_loss: 0.2009 - val_fmeasure: 0.4845
Epoch 6/50
25569/25569 [==============================] - 25s 969us/step - loss: 0.0878 - fmeasure: 0.8297 - val_loss: 0.1654 - val_fmeasure: 0.6429
Epoch 7/50
25569/25569 [==============================] - 2

In [47]:
earlyStp = EarlyStopping(patience=5, monitor='val_fmeasure', mode = 'max', verbose=1)
modelChePnt = ModelCheckpoint('bi_GRU_attention.hdf5', save_best_only=True, verbose=1, monitor='val_fmeasure', mode='max')
# f1 = f1Evaluation(validation_data=(X_val, y_val), interval=1)

hist = model_gru.fit(x = X_tra, y = y_tra, epochs = epochs, validation_data=[X_val, y_val], batch_size=batch_size, callbacks=[earlyStp, modelChePnt])
model_gru.load_weights('bi_GRU_attention.hdf5')

Train on 25569 samples, validate on 6393 samples
Epoch 1/50
25569/25569 [==============================] - 12s 475us/step - loss: 0.1155 - fmeasure: 0.7906 - val_loss: 0.1784 - val_fmeasure: 0.5627
Epoch 2/50
25569/25569 [==============================] - 11s 422us/step - loss: 0.0797 - fmeasure: 0.8401 - val_loss: 0.1863 - val_fmeasure: 0.4600
Epoch 3/50
25569/25569 [==============================] - 11s 430us/step - loss: 0.0593 - fmeasure: 0.8767 - val_loss: 0.2003 - val_fmeasure: 0.6663
Epoch 4/50
25569/25569 [==============================] - 11s 428us/step - loss: 0.0427 - fmeasure: 0.9122 - val_loss: 0.1979 - val_fmeasure: 0.5667
Epoch 5/50
25569/25569 [==============================] - 11s 421us/step - loss: 0.0363 - fmeasure: 0.9252 - val_loss: 0.1711 - val_fmeasure: 0.6136
Epoch 6/50
25569/25569 [==============================] - 11s 442us/step - loss: 0.0254 - fmeasure: 0.9503 - val_loss: 0.2399 - val_fmeasure: 0.7037
Epoch 7/50
25569/25569 [==============================] -

In [75]:
#ensembling
y_pred_lstm = model_lstm.predict(X_test, batch_size=1024, verbose=1)
y_pred_gru = model_gru.predict(X_test, batch_size=1024, verbose=1)

y_pre = (y_pred_lstm + y_pred_gru)/2

17197/17197 [==============================] - 2s 97us/step


In [49]:
model_gru.load_weights('bi_GRU_attention.hdf5')
y_pre = model_gru.predict(X_test, batch_size=1024, verbose=1)

17197/17197 [==============================] - 2s 126us/step


In [69]:
model_lstm.load_weights('bi_LSTM_attention.hdf5')
y_pre = model_lstm.predict(X_test, batch_size=1024, verbose=1)

17197/17197 [==============================] - 5s 270us/step


In [83]:
submission = pd.read_csv('sample_submission_3Mm4cJo.csv')
submission['id'] = test.id
submission['label'] = y_pre/5

In [84]:
thres = 0.5
submission['label'].loc[submission['label']>=thres]=1
submission['label'].loc[submission['label']<thres]=0

/home/paperspace/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [85]:
submission['label'] = submission.label.astype('int')
submission.head()

,id,label
0,31963,0
1,31964,1
2,31965,0
3,31966,0
4,31967,0


In [86]:
submission.label.value_counts()

0    16272
1      925
Name: label, dtype: int64

In [80]:
submission.to_csv('sub.csv', index=False)

In [119]:
test_comments[1]

' white supremacists want everyone see new  birds movie  heres'

In [93]:
y_pred[y_pred>=0.1]=0
y_pred[y_pred<=0.1]=1
Counter(y_pred.astype('int')[:, 0])

Counter({1: 17197})

In [89]:
data = train[0:170]
gen = data['tweet'].values
gen = tokenize.texts_to_sequences(gen)
gen = sequence.pad_sequences(gen, maxlen=maxlen, padding='post')

pred = np.round(model_lstm.predict(gen))

data.loc[(pred.T != data['label'].values).reshape(-1)].head()

,id,label,tweet
17,18,1,retweet if you agree!
52,53,0,off to concelebrate at the #albanpilgrimage fo...
53,54,0,@user let the scum-baggery begin.....
68,69,1,ð©the white establishment can't have blk fol...
114,115,1,@user why not @user mocked obama for being bla...


In [1]:
num = 68
pred[num]

NameError: name 'pred' is not defined

In [2]:
data.iloc[num].tweet

NameError: name 'data' is not defined

In [3]:
embeddings_index.get('scum-baggery')

NameError: name 'embeddings_index' is not defined